In [1]:
using Perla1, Parameters, Test, LinearAlgebra, DifferentialEquations

┌ Info: Recompiling stale cache file C:\Users\Chiyoung Ahn\.julia\compiled\v1.0\Perla1\nyjg2.ji for Perla1 [c0b4307c-ed70-5091-a49a-e521e9681a02]
└ @ Base loading.jl:1184
┌ Warning: Package Perla1 does not have DiffEqOperators in its dependencies:
│ - If you have Perla1 checked out for development and have
│   added DiffEqOperators as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Perla1
└ Loading DiffEqOperators into Perla1 from project dependency, future warnings for Perla1 are suppressed.


In [2]:
# sanity check fcns
# get Q in matrix form for sanity check
function get_Q_matrix(N, μ, θ, θ_d, f0)
    dl = fill(μ, N)
    d = collect(-μ.-(N:-1:0).*(θ/N))
    du = collect((N:-1:1).*(θ/N))
    Q_basis = LinearAlgebra.Tridiagonal(dl,d,du)

    function Q(a)
        Q_basis[1,1] = -(θ + θ_d*(1-f0(a)))
        Q_basis[1,2] = θ + θ_d*(1-f0(a))
        return Q_basis
    end 

    return Q
end

# solve model with matrix Q
function solve_transition_dynamics_matrix(Q_matrix, f_0, T)
    # solve transition dynamics given 
    # Q; N by N matrix generator
    # f_0; N vector of initial distribution
    # T; Float64 terminal time
    df(f,p,a) = Q_matrix(a)' * f
    prob = DifferentialEquations.ODEProblem(df,f_0,(0.0,T))
    return solve(prob);
end

function sanity_check_dynamics(Q, params, f_0) 
    # solve dynamics
    sol_count = solve_transition_dynamics(Q, params, f_0, T)

    # solve dynamics, using matrix for benchmark
    Q_matrix = get_Q_matrix(params.N, params.μ, params.θ, params.θ_d, params.f0)
    sol_count_matrix = solve_transition_dynamics_matrix(Q_matrix, f_0, T)

    # average product awareness
    f_count(a) = dot(0:N, sol_count(a)) 
    f_count_matrix(a) = dot(0:N, sol_count_matrix(a)) # average awareness

    # check if f is a probability distribution for all t in (0, T)
    @test all(sum.(sol_count.(ts)) .≈ 1.0)

    # check if there is no forgetting, 
    # i.e., f_count is increasing.
    @test all(diff(f_count.(ts)) .> 0)

    # check if solutions are close to the ones based on matrix
    # (if Q is time-variant there are numerical errors, be more generous)
    f0s = (params.f0).(ts)
    is_Q_time_variant = any(y -> y != first(f0s), f0s)
    if (is_Q_time_variant)
        @test f_count.(0:0.1:T) ≈ f_count_matrix.(0:0.1:T) atol=1e+1
    else
        @test f_count.(0:0.1:T) ≈ f_count_matrix.(0:0.1:T) atol=1e-4
    end
end

sanity_check_dynamics (generic function with 1 method)

In [3]:
N = 2
μ = 0.0
θ = 0.06 # baseline parameter from Perla16 (Appendix E.4)
θ_d = 0.0 # time-invariant transition matrix
T = 1
ts = range(0.0, stop = T, length = 50)
f0(a) = 0.5 # time-invariant transition matrix
cohort_single = (N,)

params_base = @with_kw (N = N, μ = μ, θ = θ, θ_d = θ_d, f0 = f0, cohorts = cohort_single) # base parameters
params = params_base()
f_0 = [1.0; fill(0.0, N)]

3-element Array{Float64,1}:
 1.0
 0.0
 0.0

In [4]:
# time-variant dynamics, with single cohort
function Q_a!(df, f, p, t)
    @unpack μ, θ, θ_d, f0, cohorts = p
    N_1 = cohorts[1]
    e_1 = CartesianIndex((1,))
    current_cohort = 1 # single cohort case
    
    f = reshape(f, (cohorts.+1))
    df = reshape(df, (cohorts.+1))
    
    df[1] = -(θ + θ_d*(1-f0(t)))*f[1] + μ*f[2]
    for i in CartesianIndices(f)
        if (i[current_cohort] > 1 && i[current_cohort] <= N_1)
            i_previous = i - e_1
            i_forward = i + e_1
            df[i] = θ*((N+2-i[current_cohort])/N)*f[i_previous] - 
                    (μ+θ*((N+1-i[current_cohort])/N))*f[i] + 
                    μ*f[i_forward]
        end
    end
    df[2] = (θ + θ_d*(1-f0(t)))*f[1] - (μ+θ*((N-1)/N))*f[2] + μ*f[3]
    df[end] = (θ/N)*f[N] - μ*f[N+1]
    
    f = reshape(f, (N_1+1,))   
    df = reshape(df, (N_1+1,))
end

Q_a! (generic function with 1 method)

In [5]:
sanity_check_dynamics(Q_a!, params, f_0) 

Test Passed